In [43]:
import pandas as pd
from dotenv import load_dotenv
import os
from openai import OpenAI
import json

load_dotenv()

API_KEY = os.getenv("OPENAI_TOKEN")

lote_dataframes = {
    "20231110": pd.read_excel('/home/enzo/datasets/Secadero_Lote_20231110.xlsx'),
    "20231207": pd.read_excel('/home/enzo/datasets/Secadero_Lote_20231207.xlsx'),
    "20240104": pd.read_excel('/home/enzo/datasets/Secadero_Lote_20240104.xlsx'),
    "20240202": pd.read_excel('/home/enzo/datasets/Secadero_Lote_20240202.xlsx'),
    "20240229": pd.read_excel('/home/enzo/datasets/Secadero_Lote_20240229.xlsx'),
    "20240412": pd.read_excel('/home/enzo/datasets/Secadero_Lote_20240412.xlsx'),
    "20240516": pd.read_excel('/home/enzo/datasets/Secadero_Lote_20240516.xlsx'),
    "20240620": pd.read_excel('/home/enzo/datasets/Secadero_Lote_20240620.xlsx'),
    "20250123": pd.read_excel('/home/enzo/datasets/Secadero_Lote_20250123.xlsx'),
}

client = OpenAI(api_key=API_KEY)

def extract_lote_data(model: str, df: pd.DataFrame, lote: str):
    """
    Extracts data from a DataFrame using OpenAI's chat completions API and returns the JSON response.

    Args:
        model (str): The name of the OpenAI model to use.
        df (pd.DataFrame): The DataFrame containing the lote data.
        lote (str): The lote number as a string.

    Returns:
        str: The JSON response from the OpenAI API.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "developer",
                "content": "You will receive texts, you must extract information as Lote, Producto, Fecha de produccion, each Pieza has date, you must include PIEZA 1, PIEZA 2, PIEZA 3, PIEZA 4, PIEZA 5 into JSON data, you must dont take into account the accents and special characters. If the Product dont have any Producto use PRODUCT NAME NOT PROVIDED."
            },
            {
                "role": "user",
                "content": f"Lote {lote} {df}"
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "pesajes_schema",
                "schema": {
                    "type": "object",
                    "properties": {
                        f"{lote}": {
                            "type": "object",
                            "properties": {
                                "Producto": {
                                    "description": "El nombre del producto",
                                    "type": "string"
                                },
                                "Fecha de produccion": {
                                    "description": "La fecha de produccion del producto",
                                    "type": "string",
                                    "format": "date"
                                },
                                "Pesajes": {
                                    "description": "Lista de pesajes del producto",
                                    "type": "array",
                                    "items": {
                                        "type": "object",
                                        "properties": {
                                            "Fecha": {
                                                "description": "La fecha del pesaje",
                                                "type": "string",
                                                "format": "date"
                                            },
                                            "Pesos": {
                                                "description": "Pesos de las piezas en la fecha del pesaje",
                                                "type": "object",
                                                "properties": {
                                                    "Pieza 1": {
                                                        "description": "Peso de la pieza 1",
                                                        "type": "string"
                                                    },
                                                    "Pieza 2": {
                                                        "description": "Peso de la pieza 2",
                                                        "type": "string"
                                                    },
                                                    "Pieza 3": {
                                                        "description": "Peso de la pieza 3",
                                                        "type": "string"
                                                    },
                                                    "Pieza 4": {
                                                        "description": "Peso de la pieza 4",
                                                        "type": "string"
                                                    },
                                                    "Pieza 5": {
                                                        "description": "Peso de la pieza 5",
                                                        "type": "string"
                                                    }
                                                },
                                                "required": ["Pieza 1", "Pieza 2", "Pieza 3", "Pieza 4", "Pieza 5"],
                                                "additionalProperties": False
                                            }
                                        },
                                        "required": ["Fecha", "Pesos"],
                                        "additionalProperties": False
                                    }
                                }
                            },
                            "required": ["Producto", "Fecha de produccion", "Pesajes"],
                            "additionalProperties": False
                        }
                    },
                    "required": [f"{lote}"],
                    "additionalProperties": False
                }
            }
        }
    )

    return response.choices[0].message.content


def create_combined_json(model: str, lote_dataframes: dict, output_file: str):
    """
    Extracts data for all lotes in lote_dataframes and saves it to a single JSON file.

    Args:
        model (str): The name of the OpenAI model to use.
        lote_dataframes (dict): A dictionary where keys are lote numbers and values are pandas DataFrames.
        output_file (str): The path to the output JSON file.
    """
    all_lotes_data = {}

    for lote, df in lote_dataframes.items():
        try:
            json_string = extract_lote_data(model, df, lote)
            lote_data = json.loads(json_string)  # Parse the JSON string into a Python dict
            all_lotes_data.update(lote_data)  # Add the lote data to the combined dictionary

        except Exception as e:
            print(f"Error processing lote {lote}: {e}")
            continue  # Skip to the next lote if there's an error

    # Save the combined data to a JSON file
    try:
        with open(output_file, "w") as f:
            json.dump(all_lotes_data, f, indent=4)  # Use indent for readability
        print(f"Combined JSON data saved to {output_file}")
    except Exception as e:
        print(f"Error saving combined JSON to file: {e}")


if __name__ == '__main__':
    model_name = "gpt-4o-2024-08-06"
    output_file_path = "/home/enzo/secadero/all_lotes.json"

    create_combined_json(model_name, lote_dataframes, output_file_path)

Combined JSON data saved to /home/enzo/secadero/all_lotes.json
